### listの動作

In [3]:
layers = [1,2,3]
layers += [5,6,7]
print(layers)

[1, 2, 3, 5, 6, 7]


### torch.Tensorの次元の増減

In [5]:
import torch
from torch import nn
from torch.nn import init

In [6]:
weight = nn.Parameter(torch.Tensor(512))
init.constant_(weight, 20)
print(weight.size())

torch.Size([512])


In [7]:
weight.unsqueeze(0).size()

torch.Size([1, 512])

In [8]:
weight.unsqueeze(0).unsqueeze(2).size()

torch.Size([1, 512, 1])

In [9]:
weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).size()

torch.Size([1, 512, 1, 1])

In [10]:
batch_size = 5
x = torch.ones((batch_size, 512, 38, 38))
x.size()

torch.Size([5, 512, 38, 38])

### 対象の変数の次元を別の変数の次元と同じにする

In [11]:
ww = weight.unsqueeze(0).unsqueeze(2).unsqueeze(3).expand_as(x)
print(ww.size())

torch.Size([5, 512, 38, 38])


In [12]:
x.pow(2).size()

torch.Size([5, 512, 38, 38])

In [13]:
x.pow(2).sum(dim=1).size()

torch.Size([5, 38, 38])

### 次元を保ったままの和算

In [14]:
x.pow(2).sum(dim=1, keepdim=True).size()

torch.Size([5, 1, 38, 38])

In [15]:
xx = x.pow(2).sum(dim=1, keepdim=True).sqrt()+1e-10
print(xx.size())
print(xx[0, :, :, :])

torch.Size([5, 1, 38, 38])
tensor([[[22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274],
         [22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274],
         [22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274],
         ...,
         [22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274],
         [22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274],
         [22.6274, 22.6274, 22.6274,  ..., 22.6274, 22.6274, 22.6274]]])


In [16]:
wx = xx * ww
print(wx.size())
print(wx)

torch.Size([5, 512, 38, 38])
tensor([[[[452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          ...,
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483]],

         [[452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          ...,
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483],
          [452.5483, 452.5483, 452.5483,  ..., 452.5483, 452.5483, 452.5483]],

 

### list or np.ndarray からtorch.tensorを作る

In [17]:
torch.tensor([[1.0, -1.0], [1.0, -1.0]])

tensor([[ 1., -1.],
        [ 1., -1.]])

In [18]:
import numpy as np
torch.tensor(np.array([[1,2,3],[4,5,6]]))

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)

torch.tensor()はいつもデータをコピーする

### データ・タイプを指定してtensorを作る

In [19]:
torch.zeros([2,4], dtype=torch.int32)

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0]], dtype=torch.int32)

### torch.tensorのスライシング

In [20]:
x = torch.tensor([[1,2,3],[4,5,6]])
print(x[0][1])

tensor(2)


In [21]:
x[0][1] = 8
print(x)

tensor([[1, 8, 3],
        [4, 5, 6]])


### torch.tensorからpython定義の数値を抽出する

In [22]:
x = torch.tensor([[1]])
print(x)
print(x.item())

tensor([[1]])
1


In [23]:
x = torch.tensor(2.5)
print(x)
print(x.item())
print(x.layout)

tensor(2.5000)
2.5
torch.strided


### torch.tensorは自動微分OFFの状態で生成される

In [24]:
x = torch.tensor([[1., -1.], [1., 1.]])
print("requires_grad: ", x.requires_grad)
print(x.layout)

requires_grad:  False
torch.strided


In [25]:
x = torch.tensor([[1., -1.], [1., 1.]], requires_grad=True)
print("requires_grad: ", x.requires_grad)
print("x.grad: ", x.grad)
out = x.pow(2).sum()
print("out: ", out)
out.backward()
print("x.grad after out.backward(): \n", x.grad)

requires_grad:  True
x.grad:  None
out:  tensor(4., grad_fn=<SumBackward0>)
x.grad after out.backward(): 
 tensor([[ 2., -2.],
        [ 2.,  2.]])


現在の実装では, torch.tensorはメモリのオーバヘッドが大きいので、小さな複数のtorch.tensorを運用するのはメモリ消費が激しくなる.

### dtypeとdeviceを引き継きながら、既存データからデータをコピーしたtorch.tensorを作る

In [26]:
tensor = torch.ones((2,), dtype=torch.int8)
data = [[0,1],[2,3]]
print(tensor.new_tensor(data))
print(tensor.clone())
print(tensor.clone().detach())

tensor([[0, 1],
        [2, 3]], dtype=torch.int8)
tensor([1, 1], dtype=torch.int8)
tensor([1, 1], dtype=torch.int8)


### requires_gradを変更する

In [27]:
x = torch.tensor([[0,1],[2,3]], dtype=torch.float32)
print(x.requires_grad)
x.requires_grad_(True) # 浮動小数点型のみ勾配変数を持つ
print(x.requires_grad)

False
True


new_full(size,fill_value,dtype=None,device=None,requires_grad=False) -> Tensor

In [30]:
# dtypeとdeviceを継承
tensor = torch.ones((2,), dtype=torch.float64)
print(tensor)
print(tensor.device)
new_tensor = tensor.new_full((3,4), fill_value=3.141592)
print(new_tensor)
print(new_tensor.device)

tensor([1., 1.], dtype=torch.float64)
cpu
tensor([[3.1416, 3.1416, 3.1416, 3.1416],
        [3.1416, 3.1416, 3.1416, 3.1416],
        [3.1416, 3.1416, 3.1416, 3.1416]], dtype=torch.float64)
cpu


new_empty(size,dtype=None,device=None,requires_grad=False) -> Tensor

In [31]:
tensor = torch.ones(())
tensor = tensor.new_empty((2,3))
print(tensor)
print(tensor.dtype)

tensor([[6.7444e-10, 1.6925e+22, 1.0991e-05],
        [8.5828e-07, 1.3517e+22, 2.6977e-09]])
torch.float32


new_ones(size,dtype=None,device=None,requires_grad=False) -> Tensor

In [32]:
tensor = torch.tensor((), dtype=torch.int32)
tensor.new_ones((2,3))

tensor([[1, 1, 1],
        [1, 1, 1]], dtype=torch.int32)

new_zero(size,dtype=None,device=None,requires_grad=False) -> Tensor

In [33]:
tensor = torch.tensor((), dtype=torch.float64)
tensor.new_zeros((2,3))

tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

### torch.tensorがGPUメモリにあるか否かのチェック

In [34]:
tensor = torch.tensor((), dtype=torch.float32)
print(tensor)
print(tensor.is_cuda)
print(tensor.device)

tensor([])
False
cpu


### quantizedか否かのチェック

In [35]:
print(tensor.is_quantized)

False


### tensorのdeviceチェック

In [36]:
print(tensor.device)

cpu


### 次元

In [37]:
tensor.dim()

1

### 転置

In [38]:
x = torch.tensor([[0,1],[2,3]], dtype=torch.int32)
print(x)
print(x.T)

tensor([[0, 1],
        [2, 3]], dtype=torch.int32)
tensor([[0, 2],
        [1, 3]], dtype=torch.int32)


### 要素数を維持しながらtensorの形状を変える。もし引数に与えた要素数が元の数値より低い場合は、元のtensorの中身は削られる

In [39]:
a = torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
y = a.clone().detach()
x = torch.tensor([0,1,2,3,4,5,6,7,8])
print("a shape", a.size())
print("x shape", x.size())
print("resized a shape", a.resize_as_(x))

c = a.clone().detach() # 元tensor:aの自動微分の関係を引き継がないためにdetach()が必要。
print("c", c)
print(c is a)

b = torch.tensor([[9,8],[6,5]])
print("b shape", b.size())
print("resized a shape", a.resize_as_(b))
print("a shape", a.size())

e = torch.tensor([[0,0],[0,0]])
print("y", y)
print("resized y", y.resize_as_(e))

a shape torch.Size([3, 3])
x shape torch.Size([9])
resized a shape tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])
c tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])
False
b shape torch.Size([2, 2])
resized a shape tensor([[0, 1],
        [2, 3]])
a shape torch.Size([2, 2])
y tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
resized y tensor([[0, 1],
        [2, 3]])


In [40]:
num_batch = 5
conf_t_label = torch.randint(0, 21,(num_batch, 8732))
print(conf_t_label.size())
print(conf_t_label)

torch.Size([5, 8732])
tensor([[ 7,  2,  7,  ...,  4,  5, 15],
        [ 1,  5,  1,  ...,  6, 11, 20],
        [15,  6, 14,  ...,  0, 11,  2],
        [ 1,  4,  1,  ..., 19,  5,  6],
        [11,  6, 16,  ..., 14, 13, 16]])


In [41]:
pos_mask = conf_t_label > 0
print(pos_mask)

tensor([[ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ..., False,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True]])


In [42]:
num_pos = pos_mask.long().sum(dim=1, keepdim=True)
print(num_pos)

tensor([[8318],
        [8316],
        [8305],
        [8294],
        [8311]])


In [43]:
loss_c = torch.randn((num_batch * 8732,))
print(loss_c.size())
print(loss_c)

torch.Size([43660])
tensor([ 0.1042,  0.2166,  0.0200,  ...,  0.8004, -1.0643,  1.3610])


In [44]:
loss_c2 = loss_c.view(num_batch, -1)
print(loss_c2.size())
print(loss_c2)

torch.Size([5, 8732])
tensor([[ 0.1042,  0.2166,  0.0200,  ..., -0.4098, -0.7046, -0.3654],
        [ 1.3358,  0.1795,  0.5782,  ...,  0.1677,  0.1480,  1.0245],
        [-0.7093,  0.0204, -0.6827,  ...,  0.6882,  0.2496,  0.8670],
        [-0.0508, -0.4019, -0.0333,  ..., -1.3569, -0.5838, -1.6940],
        [ 1.0672, -0.0114, -0.4001,  ...,  0.8004, -1.0643,  1.3610]])


In [45]:
print(loss_c2[pos_mask])
print(loss_c2[pos_mask].size())

tensor([ 0.1042,  0.2166,  0.0200,  ...,  0.8004, -1.0643,  1.3610])
torch.Size([41544])


In [46]:
loss_c3 = loss_c2.clone()
loss_c3[pos_mask] = 0
print(loss_c3.size())
print(loss_c3)

torch.Size([5, 8732])
tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.6882, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])


In [47]:
sorted_loss_c3, mapping_loss_c3_index = loss_c3.sort(dim=1, descending=True)

In [48]:
print(sorted_loss_c3)

tensor([[ 3.0382,  2.2594,  2.2516,  ..., -2.1319, -2.2270, -2.3968],
        [ 2.7377,  2.5797,  2.5308,  ..., -2.4066, -2.4149, -3.0637],
        [ 2.7430,  2.7130,  2.4176,  ..., -2.5612, -2.8135, -3.4760],
        [ 2.9681,  2.6779,  2.6597,  ..., -2.6926, -2.7973, -3.0354],
        [ 2.9549,  2.3109,  2.2647,  ..., -2.4141, -2.4368, -2.4930]])


In [49]:
print(mapping_loss_c3_index)

tensor([[ 516, 3462, 4723,  ..., 5513, 6714, 3574],
        [7089, 8580, 7688,  ..., 2263, 1112, 6523],
        [7573, 5141, 4916,  ..., 3301, 4945, 2366],
        [4381, 4456, 7522,  ..., 7819, 2700, 1801],
        [8290, 6671, 8146,  ..., 4215, 3745, 2722]])


In [50]:
sorted_mapping, loss_rank = mapping_loss_c3_index.sort(dim=1)
print(sorted_mapping)
print(loss_rank)

tensor([[   0,    1,    2,  ..., 8729, 8730, 8731],
        [   0,    1,    2,  ..., 8729, 8730, 8731],
        [   0,    1,    2,  ..., 8729, 8730, 8731],
        [   0,    1,    2,  ..., 8729, 8730, 8731],
        [   0,    1,    2,  ..., 8729, 8730, 8731]])
tensor([[8211, 8237, 8238,  ..., 8536, 8537, 8538],
        [ 205,  206,  207,  ...,  764,  772,  805],
        [1243,  206,  207,  ...,   79,  393,  433],
        [7494, 7533, 7551,  ..., 8525, 8526, 8527],
        [7888, 7937, 7945,  ..., 8520, 8521, 8522]])


In [51]:
print(loss_c3[0, loss_rank[0, 0]])

tensor(0.)


In [52]:
print(loss_c3[0, loss_rank[0, 1]])

tensor(0.)
